***간단한 제미나이3 채팅 api를 사용해 본다***

ai 스튜디오에서 사용했던 경험을 바탕으로 get code 를 이용하여 세세한 api 사용 방법을 연습한다

첫번째로 당연하겠지만 구글 api 를 깔아야 된다
pip install google-genai

api_key 의 환경변수 설정을 위하여 dotenv 패키지도 설치 해 줘야 한다
pip install python-dotenv

In [35]:
# 필요한 모듈 임포트

from google import genai                # 구글 제미니 API용 라이브러리 임포트
from google.genai import types          # 구글 제미니 API용 타입 임포트
from dotenv import load_dotenv          # .env 파일에서 환경변수 로드용 라이브러리 임포트
import os                               # 운영체제 관련 라이브러리 임포트
from PIL import Image                   # 필로우를 불러온다
import time                             # 시간 관련 라이브러리 임포트
import yt_dlp                           # 유튜브 다운로드 라이브러리 임포트

load_dotenv()                           # .env 파일에서 환경변수 로드

True

In [13]:
# 구글 제미니 API 키 설정
# genai.configure(api_key="YOUR_API_KEY_HERE")  # 여기에 실제 API 키를 입력하세요.(예전 방식임)
# api_key = genai.Client(api_key="AIzaSyAzAVTWgdgt-*************")  # 여기에 실제 API 키를 입력하세요.(최신 방식임)

# 위와 같은 방식은 api_key 가 그대로 노출되기 때문에 보안에 취약하다. 때문에 환경변수로 설정하는 방식을 권장한다.
# 때문에 아래와 같이 환경변수로 설정하는 방식을 사용한다.
# 또한 환경 변수를 지정 할 수 있는 dotenv 의 설치도 필요하다
gemini_client = genai.Client(api_key=os.getenv("gemini_api_key"))

In [14]:
# 제미나이 모델 설정
model = "gemini-3-flash-preview"  # 사용할 제미나이 플래쉬 모델 1M 토큰당 input 0.5달러, output 3.0달러
# model = "gemini-3-pro-preview"  # 사용할 제미나이 프로 모델 1M 토큰당 input 2.0달러, output 12.0달러

In [ ]:
# 콘텐츠 설정 - 사용자 입력즉 프롬프트 부분
# 이게 바뀌어서 types.Content() 로 감싸줘야 한다
content = [types.Content(role="user", parts=[types.Part(text="이 이미지를 분석 해줘")])]
# 이렇게 하면 content 가 누가 말했는지, 뭘 말했는지 구분이 된다

In [ ]:
# 자 이제 api 키와, 모델, 콘텐츠(프롬프트) 가 모두 준비 되었으니 요청을 날려보자

# 아까 만든 나 즉 gemini_client 로 요청을 날린다.
# 요청을 날릴때 model 과 content(프롬프트) 를 같이 넘겨준다
response = gemini_client.models.generate_content(model=model, contents=content)

print(response.text) # 응답 받은 텍스트 출력

In [ ]:
# 멀티모달의 성능을 확인해 보자
# 일단 이미지를 열어서 바이너리로 읽어 들인다음 ai에게 전달 해야 한다

# 이미지를 열자
with open("dance.png", "rb") as image_file:     #image_file이라고 바이너리로 png 파일을 연다
    image_bytes = image_file.read()             # image_bytes 라는 변수에 dance.png의 바이너리 데이터를 읽어서 저장

# 멀티 모달 답게 parts를 텍스트와 이미지로 같이 보낸다(구글이 자랑하는 멀티모달 기능)
image_content = [types.Content(role="user",
                               parts=[types.Part(text="이 이미지를 분석 해줘"),
                                      types.Part(inline_data=types.Blob(mime_type="image/png", data=image_bytes))
                                     ]
                              )
                ]

response = gemini_client.models.generate_content(model=model, contents=image_content)

print(response.text) # 응답 받은 텍스트 출력

In [ ]:
# 디테일하게 types 를 사용하면 role 과 parts 를 명확하게 구분해서 보낼 수 있는데... 
# 단편적인 답을 얻을때에는 꼭 이렇게 할 필요는 없다.
# pillow 를 쓰면 이미지 처리가 더 편리하다 하지만 디테일한 types 설정은 생략이 된다.
# 일단 pillow 를 설치 해보자
# pip install pillow

img = Image.open("dance.png")          # dance.png 파일을 연다 필로우가 여는것이다 무척 단순... 
simple_content = [types.Content(role="user",
                                parts=[types.Part(text="이 이미지를 분석 해줘"),
                                       types.Part(image=img)   # 이미지 부분은 그냥 필로우 이미지 객체를 넘긴다
                                      ]
                               )
                ]
response = gemini_client.models.generate_content(model=model, contents=simple_content)

print(response.text) # 응답 받은 텍스트 출력

# 이렇게 하면 에러가 난다 types.Part(image=img) 부분에서 에러가 난다 왜냐면 image 부분에 필로우 이미지 객체를 바로 넣을 수 없기 때문이다.
# 그래서 아래와 같이 image 객체를 바이트로 변환해서 넘겨줘야 한다



In [ ]:
# 자그럼 이제 아주 간단한 방법으로 이미지와 텍스트를 혼합한 프롬프트를 작성해 보자

img = Image.open("dance.png")          # dance.png 파일을 연다 필로우가 여는것이다 무척 단순... 

simple_contents = ["이 이미지를 분석 해 줘", img]   # 이미지 부분은 그냥 필로우 이미지 객체를 넘긴다 단순히

# 이렇게 하면 내부적으로 types.Content 와 types.Part 로 자동 변환이 된다
response = gemini_client.models.generate_content(model=model, contents=simple_contents)

print(response.text) # 응답 받은 텍스트 출력
# 이렇게 하면 에러가 나지 않고 정상적으로 이미지 분석이 된다.

In [ ]:
# 이제 동영상 분석도 가능한지 해 보자

# 동영상 파일을 분석하기 위해서는 ai 서버로 동영상 파일을 업로드 해야 한다.

print("동영상 파일 업로드 중...")
video = gemini_client.files.upload(file="android.mp4")   # android.mp4 파일을 업로드 한다
print("동영상 파일 업로드 완료")

# 이렇게 업로드된 동영상은 ai가 읽을수 있는 방식으로 서버에 변환되어 임시 저장된다..
# 이게 구글의 장점이다. 내부 로직은 어떻게 변환하는지 모른다.
# 대신에 ai가 읽을 수 있도록 변환하는 동안 기다려 줘야 한다. 안그러면 오류 단순한 업로드가 아님을 알 수 있다.

while video.state.name == "PROCESSING":     # 업로드된 동영상이 처리중이면
    print("동영상 파일 처리중...")             # 처리중 메시지 출력
    time.sleep(5)                           # 5초 대기
    video = gemini_client.files.get(name=video.name)  # 다시 상태를 가져온다 비디오의 상태를 다시 가져온다

if video.state.name == "FAILED":         # 만약 동영상 처리에 실패 했으면
    print("동영상 파일 처리 실패")          # 실패 메시지 출력

print("동영상 파일 처리 완료")              # 완료 메시지 출력

content_video = ["이동영상의 내용을 분석해줘, 대사도 알려주고", video]   # 동영상 파일과 텍스트 프롬프트를 같이 보낸다   

response = gemini_client.models.generate_content(model=model, contents=content_video)

print(response.text) # 응답 받은 텍스트 출력
# 이렇게 하면 동영상의 내용과 대사를 분석해 준다.

In [ ]:
# 유튜브 동영상 분석 및 요약
# 현재 유튜브 동영상의 직접적인 분석은 지원하지 않는다.
# 하지만 유튜브를 다운받아서 위 처럼 분석하는것은 얼마든지 가능하다
# 대신 엄청난 토큰이 나가게 된다.
# 예를 들어 1시간짜리 유튜브 동영상을 다운받아서 분석하면 100만 토큰이 나가게 된다.
# 플래쉬 기준으로는 저렴하지만 pro 기준으로 보면 분석면 몇천원 수준이다.
# 주의해서 사용 해야 된다.

# 유튜브 동영상 다운로드는 yt-dlp 라는 툴을 사용 한다
# pip install yt-dlp 임포트도 잊지않고 해 준다 import yt_dlp

ydl_opts = {
            'format': 'mp4',                      # mp4 포맷으로 다운로드    
            'outtmpl': 'youtube_video.mp4',       # 저장할 파일명
            'quiet': False,                        # 출력 메시지 최소화
}

try:
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        print("유튜브 동영상 다운로드 중...")
        ydl.download(['https://www.youtube.com/watch?v=5lRFax3k91Q'])  # 여기에 실제 유튜브 동영상 URL을 입력하세요
        print("유튜브 동영상 다운로드 완료")
except Exception as e:
    print(f"유튜브 동영상 다운로드 실패: {e}")

# 다운로드된 유튜브 동영상 파일 업로드 현재 쿠키 문제때문에.. 오류가 발생한다.
# 쿠키 문제를 해결하려면 별도의 쿠키파일을 만들어야 되는데 번거로우니 생략한다.

In [ ]:
# pdf 분석을 알아보자, 동영상 분석과 비슷하다.(같다고도 할 수 있다.)

print("pdf 파일 업로드 중...")
pdf_file = gemini_client.files.upload(file="test.pdf")   # android.mp4 파일을 업로드 한다
print("pdf 파일 업로드 완료")

while pdf_file.state.name == "PROCESSING":     # 업로드된 동영상이 처리중이면
    print("pdf 파일 처리중...")             # 처리중 메시지 출력
    time.sleep(2)                           # 2초 대기
    pdf_file = gemini_client.files.get(name=pdf_file.name)  # 다시 상태를 가져온다 비디오의 상태를 다시 가져온다

if pdf_file.state.name == "FAILED":         # 만약 동영상 처리에 실패 했으면
    print("pdf 파일 처리 실패")          # 실패 메시지 출력

print("pdf 파일 처리 완료")              # 완료 메시지 출력

content_pdf = ["이 pdf의 내용을 분석해줘", pdf_file]   # 동영상 파일과 텍스트 프롬프트를 같이 보낸다   

response = gemini_client.models.generate_content(model=model, contents=content_pdf)

print(response.text) # 응답 받은 텍스트 출력
# 이렇게 하면 pdf의 내용을 분석해 준다.